# Bidirectional LSTM on IMDB

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/05/03<br>
**Last modified:** 2020/05/03<br>
**Description:** Train a 2-layer bidirectional LSTM on the IMDB movie review sentiment classification dataset.

## Setup

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review


## Build the model

In [28]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")

# Embed each integer in a 128-dimensional vector (one-hot encoding)
# https://keras.io/api/layers/core_layers/embedding/
x = layers.Embedding(max_features, 128)(inputs)

# Add 2 bidirectional LSTMs
# https://keras.io/api/layers/recurrent_layers/bidirectional/
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
# x = layers.Bidirectional(layers.LSTM(64))(x)

# Add a classifier (logistic)
y = layers.Dense(64, activation="linear")(x)
outputs = layers.Dense(1, activation="sigmoid")(y)
model = keras.Model(inputs, outputs)

model.summary()


Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_6 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
bidirectional_7 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
dense_10 (Dense)             (None, None, 32)          4128      
_________________________________________________________________
dense_11 (Dense)             (None, None, 1)           33        
Total params: 2,662,977
Trainable params: 2,662,977
Non-trainable params: 0
_________________________________________________________________


## Load the IMDB movie review sentiment data

In [29]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)


25000 Training sequences
25000 Validation sequences


In [30]:
x_train.shape

(25000, 200)

In [31]:
y_train.shape

(25000,)

## Train and evaluate the model

In [32]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))


Epoch 1/2
782/782 [==============================] - 147s 188ms/step - loss: 0.5642 - accuracy: 0.7112 - val_loss: 0.5581 - val_accuracy: 0.7074
Epoch 2/2
782/782 [==============================] - 139s 177ms/step - loss: 0.5078 - accuracy: 0.7523 - val_loss: 0.5474 - val_accuracy: 0.7086


# Exercise: Try to use only one LSTM layer to train the model again.